In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", palette="husl")

%load_ext autoreload
%autoreload 2

import numpy as np
import gym
import sys
import os
if "../" not in sys.path:
    sys.path.insert(0, "../")

from algo.lstd import LSTDQ, LSTDMu, LSPI
from algo.policy import RandomPolicy2, LinearQ2
from env.simulator import * 
from util.plotting import *
from util.basis import *
from algo.fa import LinearQ3
from algo.dqn import DQN

from logger import *
setup_logging(default_level=logging.INFO)


/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
env_id = "MountainCar-v0"
env = gym.envs.make(env_id)
state_dim = env.observation_space.shape[0]
# discrete action
action_dim = 1
n_action = env.action_space.n
sim = Simulator(env, state_dim=state_dim, action_dim=action_dim)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Online mode DQN
- mountain car has reward of -1 everywhere and dqn's prediction is likely initialized around zero. It's very optimistic and therefore it will end up exploring as much as possible wherever it did not get -1's. therefore epsilon can be very low.
- the learning rate matters

In [9]:
import tensorflow as tf

# test online dqn
with tf.Graph().as_default():
    dqn = DQN(env,
               D=None,
               hiddens=[128, 64],
               learning_rate=1e-4,
               gamma=0.99,
               buffer_size=50000,
               max_timesteps=10**5,
               print_freq=100,
               layer_norm=True,
               exploration_fraction=0.0001,
               exploration_final_eps=0.0001,
               param_noise=True,
               grad_norm_clipping=10,
               buffer_batch_size=32)


In [10]:
with tf.Graph().as_default():
    dqn.train(use_batch=False)

/usr/local/lib64/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib64/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 0        |
| episodes                | 30       |
| mean 100 episode reward | -200     |
| steps                   | 5799     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 0        |
| episodes                | 60       |
| mean 100 episode reward | -199     |
| steps                   | 11734    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 0        |
| episodes                | 90       |
| mean 100 episode reward | -197     |
| steps                   | 17520    |
--------------------------------------
Saving model due to mean reward increase: None -> -197.2
--------------------------------------
| % time spent exploring  | 0        |
| episodes                | 120      |
| mean 100 episode reward | -196     |
| steps                   | 23353    |
--------------------------------------
-------

INFO:tensorflow:Restoring parameters from /tmp/tmpvwzwhii1/model


Saving model to mountaincar_model.pkl


In [ ]:
# PERcheck ?


## Batch mode DQN